# Endangered Animals and Invasive Animals

- Name:  Vadde Pavan Kumar 
- Project : E24
- Work Type: Wrangling the Data Sets
- Type Of Datasets: Open Datasets/API
- Version: Python 3.0
- Date: 12-Apr-2020

#### Data Exploration for Project Forest Friends

#### Package Installations

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
#pip insatll numpy


# Importing the packages 

In [8]:
import pandas as pd 
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
#from sodapy import Socrata
import requests
import pandas as pd
import csv
#import wikipedia
import urllib.request
import random
import string
import numpy as np


- Loading the data, the data is about the endangered species, the data is collected from the data.gov.au 
- As the data is collected from the open data sets. And the link is provided to from where the data has extracted

https://data.gov.au/dataset/ds-dga-ae652011-f39e-4c6c-91b8-1dc2d2dfee8f/details

### Data Properties:

- The data is about the Endangered species in the australia and is also given that which is provided across the states of
  of the australia
- Data which is provided is the rela time data and also is provided with the API.
- The data set is comprised of animal information like Scientific Name and common name and also refrence link to the animal   species open data base.

#### Loading the data 

In [ ]:
govt_data = pd.read_csv("20200404spcs.csv")

In [ ]:
govt_data.head() 

In [ ]:

govt_data.columns

As the data is provided with the rich information of endangered species.
    - Animalia 
    - Plante 
As we are dealing with the animal information we are jsut extracting the infromation with the regards to the animals 

In [ ]:
govt_data = govt_data[govt_data['Kingdom'] == "Animalia"]

In [ ]:
govt_data.reset_index(drop=True, inplace=True)

In [ ]:
govt_data.shape

In [ ]:
govt_data.head()

##### created the new dataframe and with the state columns 
- removing the all the different columns and making one column, with all the states included in that.


In [ ]:
cols = ['ACT','NSW','NT','QLD','SA','TAS','VIC','WA']

In [ ]:
df = govt_data[['ACT','NSW','NT','QLD','SA','TAS','VIC','WA']]

In [ ]:
s = np.where(df == "Yes", ['{}'.format(x) for x in df.columns], '')
regions = pd.Series([''.join(x).strip(',') for x in s], index=df.index)
regions = s.tolist()

for j in regions:
    while '' in j:
        j.remove('')

In [ ]:
govt_data['states'] = regions

In [ ]:
govt_data.head()

### Data for australian capital teritory 

- Working on the animal species data for the australian capital teritory
- Where the data is collected from data.act.gov.au and the data is extracted using the API
- Dataset has the information about all the endangered animals and also provied with the map information where actually the animal resides 

Source: www.data.act.gov.au/e9ux-7djy

In [ ]:

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.data.act.gov.au", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(www.data.act.gov.au,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("e9ux-7djy", limit=50000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.columns

- Extracting only the necessary columns from the dataset
  - common name 
  - Scientific name 
  - postions 

In [ ]:
results_df = results_df[['commonname', 'scientificname','location_1']]

###### Storing the information into the new file 

In [ ]:
results_df.to_csv('ACT_animals.csv')

In [ ]:
results_df

### Working on the data for melbourne 

Data is extarcted from the API which is pen source and data is provided for the animals across the melbourne.

Source : data.melbourne.vic.gov.au/a945-pqqr

In [ ]:

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.melbourne.vic.gov.au", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.melbourne.vic.gov.au,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("a945-pqqr", limit=50000)

# Convert to pandas DataFrame
mel_df = pd.DataFrame.from_records(results)

##### Selecting only the animala Kingdom data 

In [ ]:
mel_df = mel_df[mel_df['state_parsed'] == "ANIMALIA"]

In [ ]:
mel_df.reset_index(drop=True, inplace=True)

In [ ]:
mel_df = mel_df[['ibra_region_parsed','collector','basis_of_record_processed','taxon_identification_issue','latitude_processed','longitude_processed']]

###### Data is stored into the CSV files 

In [ ]:
mel_df.to_csv('vic_animals.csv')

In [ ]:
mel_df

In [ ]:
mel_df = mel_df.rename(columns={'ibra_region_parsed': 'animal_type', 'basis_of_record_processed': 'scientific_name','taxon_identification_issue' : 'common_name', 'latitude_processed':'latitude','longitude_processed' : 'longitude'})

In [ ]:
mel_df.head()

### Extracting the data

## Main Code

##### Function for extracting the content from the HTML Page  - Called function

In [ ]:
def service(l1,text):
    l1 = l1
    habitat = []
    for i in range(len(l1)):
        count = 1
        if l1[i].strip() == text:
            while count <= 2:
                if l1[i].strip() == "Top":
                    count = count + 1
                habitat.append(l1[i].strip())
                i = i + 1
    while '' in habitat:
        habitat.remove('')
    habitat = habitat[2:-2]
    return(habitat)

### Main Function  - Called and Calling Function

In [ ]:
def extract(url):
    url = url
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    data = []
    blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    # there may be more elements you don't want, such as "style", etc.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)

    l1 = output.strip().split('\n')
    services = ["Habitat","Feeding","Threats","Illustrations","Descriptions"]
    for ser in services:
        data.append(service(l1,ser))
   
    return(data)

##### creating the new columns in the dataframe 

In [ ]:
govt_data['habitat'] = ""
govt_data['feeding'] = ""
govt_data['threat'] = ""
govt_data['image'] = ""
govt_data['Description'] = ""

In [ ]:
## example
#extract("http://www.environment.gov.au/cgi-bin/sprat/public/publicspecies.pl?taxon_id=26")

#### Main Function Call -- Calling function 

In [ ]:
profiles = govt_data["Profile"]
for i in range(len(profiles)):
    profile = extract(profiles[i])
    govt_data['habitat'][i] = profile[0]
    govt_data['feeding'][i] = profile[1]
    govt_data['threat'][i] = profile[2]
    govt_data['image'][i] = [x for x in list(set(profile[3])) if x.startswith('Google')]
    govt_data['Description'][i] = profile[4][0] if len(desc) > 0 else profile[4]

In [ ]:
govt_data.head()

In [ ]:
govt_data.columns

In [ ]:
#cols = ['Scientific Name', 'Common Name','Threatened status','Listed SPRAT TaxonID','Kingdom','Class','Profile',
 #      'Family','states','habitat', 'feeding', 'threat','image']

#### Picking up only the useful columns and writing to the csv file

In [ ]:
main_df = govt_data[['Scientific Name', 'Common Name','Threatened status','Listed SPRAT TaxonID','Kingdom','Class','Profile','Family','states','habitat', 'feeding', 'threat','image']]

In [ ]:
main_df.head()

In [ ]:
main_df.to_csv('endangered_animals.csv')

### Threatened Fauna From EPBC  Dataset

In [81]:
fauna = pd.read_csv("Threatened_Fauna_EPBC_2018.csv")

C:\Users\V.PAVAN\Anaconda3\envs\rstudio\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (26,27,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
fauna.columns

Index(['Record ID', 'Catalogue Number', 'Taxon Concept GUID',
       'Scientific Name - original', 'Vernacular name - original',
       'Scientific Name', 'Taxon Rank', 'Vernacular name', 'Kingdom', 'Phylum',
       'Class', 'Order', 'Family', 'Genus', 'Species', 'Subspecies',
       'Data Resource ID', 'Data Resource Name', 'Institution ID',
       'Institution', 'Collection ID', 'Collection', 'Licence',
       'Institution Code', 'Collection Code', 'Locality',
       'Latitude - original', 'Longitude - original',
       'Geodetic datum - original', 'Latitude', 'Longitude',
       'Coordinate Precision', 'Coordinate Uncertainty in Metres',
       'Country - parsed', 'State - parsed', 'Local Government Areas 2011',
       'IMCRA 4 Regions', 'IBRA 7 Regions', 'Minimum elevation in meters',
       'Maximum elevation in meters', 'Minimum depth in meters',
       'Maximum depth in meters', 'Individual count', 'Collector', 'Year',
       'Month', 'Day', 'Event Date - parsed', 'Verbatim even

In [83]:
fauna = fauna[fauna['Country - parsed'] == "Australia"]

In [ ]:
#cols = ['Scientific Name',  'Vernacular name','Class', 'Family', 'Latitude', 'Longitude','Country - parsed', 'State - parsed', 'Local Government Areas 2011','IMCRA 4 Regions', 'IBRA 7 Regions', 'Individual count']

In [84]:
fauna = fauna[['Scientific Name',  'Vernacular name','Class', 'Family', 'Latitude', 'Longitude','Country - parsed', 'State - parsed', 'Local Government Areas 2011','IMCRA 4 Regions', 'IBRA 7 Regions', 'Individual count']]

In [85]:
fauna[['IMCRA 4 Regions','IBRA 7 Regions']] = fauna[['IMCRA 4 Regions','IBRA 7 Regions']].fillna('')

In [86]:
fauna['Region'] = fauna['IMCRA 4 Regions'] + fauna['IBRA 7 Regions']

In [87]:
del fauna['IMCRA 4 Regions']
del fauna['IBRA 7 Regions']

In [89]:
fauna.shape

(361170, 11)

In [90]:
fauna = fauna.drop_duplicates(subset='Scientific Name', keep="last")

In [91]:
fauna.reset_index(drop=True, inplace=True)

In [92]:
fauna 

,Scientific Name,Vernacular name,Class,Family,Latitude,Longitude,Country - parsed,State - parsed,Local Government Areas 2011,Individual count,Region
0,Mathewsoconcha grayi ms,NaN,Gastropoda,Euconulidae,-9.568370,143.649120,Australia,Queensland,NaN,NaN,Northeast Shelf Transition
1,Dasyornis (Dasyornis) brachypterus monoides,NaN,Aves,Dasyornithidae,-28.816000,152.983000,Australia,New South Wales,Richmond Valley (A),NaN,South Eastern Queensland
2,Pseudococcus markharveyi,NaN,Insecta,Pseudococcidae,-34.400000,118.300000,Australia,Western Australia,Gnowangerup (S),1,Esperance Plains
3,Lepidodactylus listeri,Christmas Island Gecko,Reptilia,Gekkonidae,NaN,NaN,Australia,Western Australia,NaN,NaN,
4,Cyrtodactylus sadleiri,NaN,Reptilia,Gekkonidae,NaN,NaN,Australia,Western Australia,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...
180,Calidris (Calidris) tenuirostris,Great Knot,Aves,Scolopacidae,-25.958166,153.007317,Australia,Queensland,NaN,6,Central Eastern Shelf Transition
181,Mixophyes fleayi,Fleay's Frog,Amphibia,Myobatrachidae,-28.200000,153.100000,Australia,Queensland,Scenic Rim (R),NaN,South Eastern Queensland
182,Dasyornis (Dasyornis) brachypterus,Eastern Bristlebird,Aves,Dasyornithidae,-35.100000,150.600000,Australia,New South Wales,NaN,NaN,Sydney Basin
183,"Calidris canutus ( (Linnaeus), 1758) canutus",Huahou,Aves,Scolopacidae,-34.362220,137.992780,Australia,South Australia,Yorke Peninsula (DC),320,Eyre Yorke Block


In [ ]:
govt_data.columns

## shape file dataset

In [111]:
new_df = pd.read_csv("endangered_animals.csv") 

In [ ]:
shape_file = gpd.read_file("snes_public_grids_08Aug2019_shapefile/snes_species_combined.shp")

In [ ]:
shape_file.columns

In [ ]:
suburb = gpd.read_file("vic_locality_polygon_shp/VIC_LOCALITY_POLYGON_shp.shp")

In [ ]:
shape_file['lower_name'] = shape_file['sci_name'].str.lower()

### Merging for Endangered Species

In [ ]:
#Create lowercase for all the scientific names
new_df['lower_name'] = new_df['Scientific Name'].str.lower()

#Performing the inner join
locations = pd.merge(new_df[['lower_name','Common Name','Threatened status','Profile']],shape_file[['lower_name','listed_id','sci_name', 'threatened','tax_class', 'tax_phylum', 'regions', 'attributio', 'sprat_prof', 'geometry']],on='lower_name',how='inner')

In [ ]:
locations.to_csv("map_data.csv")

### Merging for the ACT Data 

In [ ]:
results_df.head()

In [ ]:
results_df['lower_name'] = results_df['scientificname'].str.lower()

#Performing the inner join
act_locations = pd.merge(results_df[['lower_name','location_1']],shape_file[['lower_name','listed_id','sci_name', 'threatened','tax_class', 'tax_phylum', 'regions', 'attributio', 'sprat_prof', 'geometry']],on='lower_name',how='inner')

In [ ]:
act_locations.to_csv("act_locations.csv")

### Merging for Melbourne Data 

In [ ]:
mel_df.head()

In [98]:
mel_df['lower_name'] = mel_df['scientific_name'].str.lower()

#Performing the inner join
mel_locations = pd.merge(mel_df[['lower_name','animal_type','latitude','longitude']],shape_file[['lower_name','listed_id','sci_name', 'threatened','tax_class', 'tax_phylum', 'regions', 'attributio', 'sprat_prof', 'geometry']],on='lower_name',how='inner')

### Merging for Fauna Data

In [99]:
fauna['lower_name'] = fauna['Scientific Name'].str.lower()

#Performing the inner join
fauna_locations = pd.merge(fauna[['lower_name','State - parsed','Class','Latitude','Longitude']],shape_file[['lower_name','listed_id','sci_name', 'threatened','tax_class', 'tax_phylum', 'regions', 'attributio', 'sprat_prof', 'geometry']],on='lower_name',how='inner')

In [101]:
fauna_locations.to_csv("fauna_locations.csv")

##### Region specific -- Suburb information 

In [ ]:
from shapely.geometry import shape, mapping, Polygon, MultiPolygon, Point

In [ ]:
new_df = gpd.GeoDataFrame(locations,geometry = "geometry")
new_df.crs = {'init':'epsg:4326'}

In [ ]:
outdf = gpd.GeoDataFrame(columns=new_df.columns)
for idx,row in new_df.iterrows():
    #print(row)
    if type(row.geometry) == MultiPolygon:
        multdf = gpd.GeoDataFrame(columns = new_df.columns)
        recs = len(row.geometry)
        multdf = multdf.append([row]*recs,ignore_index=True)
        for geom in range(recs):
            multdf.loc[geom,'geometry'] = row.geometry[geom]
        outdf = outdf.append(multdf,ignore_index=True)
    if type(row.geometry) == Polygon:
        outdf = outdf.append(row,ignore_index=True)
           

In [ ]:
new_df=gpd.GeoDataFrame(outdf,geometry='geometry')#adding the point objects as a column to the dataframe.
new_df.crs={'init' :'epsg:4326'}


In [ ]:
new_df.head()

In [ ]:


#Getting only the suburb name and the geometry of from the dataframe.
sdf = gpd.GeoDataFrame(suburb)
sdf1=sdf[['VIC_LOCA_2','geometry']]
sdf1.set_geometry('geometry')
sdf1.crs={'init' :'epsg:4326'}#Setting up the coordinate refernce system.
sdf1.head()#sdf1 is the suburb with shape files.


df_suburb=gpd.sjoin(new_df,sdf1,how='left',op='within')
df=pd.DataFrame(df_suburb)#Converting the geodataframe to the pandas dataframe for further manipulations
df=df.drop(['index_right'],axis=1)
df.rename(columns={'VIC_LOCA_2':'suburb'},inplace=True)#renaming the columns
df.head()

In [ ]:
len(df.dropna(subset=["suburb"])["lower_name"].unique())

##### Ifrmae KMV Data 

In [241]:
from IPython.display import HTML

In [110]:

HTML('<iframe src="https://www.google.com/maps/d/embed?mid=1_XeEGRPzee57Rn0wNssbgm5C6aVUOdSs" width="800" height="480"></iframe>')

In [242]:
HTML('<iframe src="https://www.google.com/maps/d/embed?mid=1Lgbb-MCcvWO9rFy9aWizA6JdNoPlq4Hw" width="800" height="480"></iframe>')

C:\Users\V.PAVAN\Anaconda3\envs\rstudio\lib\site-packages\IPython\core\display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


##### Working on the shape file 

In [145]:
sci_names = list(shape_file['sci_name'].unique())

In [154]:
shape_file.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'listed_id', 'map_id',
       'sci_name', 'comm_name', 'threatened', 'migratory', 'pres_rank',
       'category', 'extracted', 'tax_group', 'tax_family', 'tax_order',
       'tax_class', 'tax_phylum', 'tax_king', 'other_ids', 'cell_size',
       'regions', 'attributio', 'sprat_prof', 'geometry', 'lower_name'],
      dtype='object')

In [150]:
new_file = shape_file.drop_duplicates(subset=['sci_name'])

In [160]:
new_file = new_file[['listed_id','sci_name', 'comm_name', 'threatened','migratory','tax_group', 'tax_family','tax_order','tax_class','regions','sprat_prof', 'geometry']]

In [167]:
new_file.to_csv("species_shape.csv")

In [ ]:
species_file = new_file[new_file['tax_group'] != "flora"]

In [ ]:
species_file.reset_index(drop=True, inplace=True)

In [172]:
del species_file['Unnamed: 0']

In [ ]:
species_file['habitat'] = ""
species_file['feeding'] = ""
species_file['threat'] = ""
species_file['image'] = ""
species_file['Description'] = ""

In [ ]:
profiles = species_file["sprat_prof"]
for i in range(len(profiles)):
    profile = extract(profiles[i])
    species_file['habitat'][i] = profile[0]
    species_file['feeding'][i] = profile[1]
    species_file['threat'][i] = profile[2]
    species_file['image'][i] = [x for x in list(set(profile[3])) if x.startswith('Google')]
    species_file['Description'][i] = profile[4][0] if len(desc) > 0 else profile[4]

In [ ]:
species_file.to_csv("species_file.csv")

### Image Handling:

In [ ]:
def get_comm_images(name):
    except_list = []
    try:
        comm_page = wikipedia.page(name)
    except:
        print(name+' '+'created comm error')
        return except_list
    comm_images=[]
    for value in list(comm_page.images):
        if 'jpg' in value:
            comm_images.append(value)
        else:
            continue
    return comm_images
    
def comm_split(com):
    
    if ',' in com:
        com_names = com.split(',')
        # count = len(com_names)
        mid_images = []
        for midimg in com_names:
            check_mid = get_comm_images(midimg)
            if len(check_mid) > 0:
                return check_mid
            else:
                continue
        return mid_images
    else:
        return(get_comm_images(com))


def get_images(x):
    sci_name = x['Scientific_Name']
    comm_name = x['Common_Name']
    exception_list = []
    sci_exflag = 0
    sci_images_return = []
    try:
        wiki_page_sci = wikipedia.page(sci_name) #Get the page for the sci_name
        sci_images_list = list(wiki_page_sci.images)
        for value in sci_images_list:
            
            if 'jpg' in value:
                sci_images_return.append(value)
            else:
                continue

    except:
        sci_exflag = 1
        print(sci_name+' '+'Created an sci_exception')


    if sci_exflag==1 or len(sci_images_return)==0:
        common_images_return = comm_split(comm_name)
        return common_images_return
    
    if sci_exflag==0:
        return sci_images_return
    else:
        return exception_list

if __name__ == "__main__":
    fname = input('Enter filename...')
    data = pd.read_csv(fname)
    data['images'] = data.apply(lambda x: get_images(x),axis=1)
    data.to_csv('images_split_edit_animals.csv',index=False)
    print('image search done!!=:)')
    

####  check for similarities in the data:  

In [243]:
dummy = pd.read_csv("endangered_animals.csv")

In [337]:
dummy.columns

Index(['sci_name', 'Common Name', 'Threatened status', 'Listed SPRAT TaxonID',
       'Class', 'Profile', 'Family', 'states', 'habitat', 'feeding', 'threat',
       'image', 'Description'],
      dtype='object')

'Scientific Name', 'Common Name', 'Threatened status', 'SPART ID',
       'Class', 'Family', 'Description', 'Habitat', 'Feeding', 'Threats','Regions'
       'Image',  'Profile'

In [336]:
del dummy['Kingdom']

In [360]:
Animals = pd.DataFrame()

In [456]:
Animals['Regions'] = dummy['states']

In [470]:
Animals.head()

,SPART_ID,Scientific_Name,Common_Name,Threatened_Status,Class,Family,Description,Habitat,Feeding,Threats,Regions,Image,Profile,Lat_Lon
0,22,Neophoca cinerea,"Australian Sea-lion, Australian Sea Lion",Vulnerable,Mammalia,Otariidae,"The Australian Sea-lion has a blunt snout, wit...",Australian Sea-lions use a wide variety of hab...,Australian Sea-lions feed on a wide variety of...,"Historic hunting and sealingHistorically, the ...","['SA', 'WA']",Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,NA
1,26,Mirounga leonina,Southern Elephant Seal,Vulnerable,Mammalia,Phocidae,The Southern Elephant Seal is a large marine m...,To breed and moult the Southern Elephant Seal ...,Southern Elephant Seals feed mainly on cephalo...,Past threats to the Southern Elephant Seal inc...,"['SA', 'TAS']",Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,NA
2,34,Balaenoptera borealis,Sei Whale,Vulnerable,Mammalia,Balaenopteridae,Sei whales are dark grey or blue-grey on their...,The Australian Antarctic waters are important ...,Sei whales feed intensively between the Antarc...,The Action Plan for Australian Mammals 2012 ...,"['NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA']",Google Images http://e-info.org.tw/topic/whale...,http://www.environment.gov.au/cgi-bin/sprat/pu...,NA
3,36,Balaenoptera musculus,Blue Whale,Endangered,Mammalia,Balaenopteridae,Both the Antarctic blue whale and pygmy blue w...,Blue whale habitat is variable between the two...,"In Antarctic waters, the blue whale feeds on k...",Several anthropogenic threats have been identi...,"['NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']",Google Images http://www.yoto98.noaa.gov/books...,http://www.environment.gov.au/cgi-bin/sprat/pu...,NA
4,37,Balaenoptera physalus,Fin Whale,Vulnerable,Mammalia,Balaenopteridae,Fin whales are dark grey to brownish black dor...,The Australian Antarctic waters are important ...,Fin whales feed intensively in high latitudes ...,The Action Plan for Australian Mammals 2012 ...,"['NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA']",Google Images http://www.photolib.noaa.gov/ani...,http://www.environment.gov.au/cgi-bin/sprat/pu...,NA


In [346]:
dummy1 = pd.read_csv("species_file.csv")

In [412]:
Animals.columns

Index(['SPART_ID', 'Scientific_Name', 'Common_Name', 'Threatened_Status',
       'Class', 'Family', 'Description', 'Habitat', 'Feeding', 'Threats',
       'Regions', 'Image', 'Profile'],
      dtype='object')

In [413]:
dummy1.columns

Index(['listed_id', 'sci_name', 'comm_name', 'threatened', 'migratory',
       'tax_family', 'tax_class', 'regions', 'sprat_prof', 'geometry',
       'habitat', 'feeding', 'threat', 'image', 'Description'],
      dtype='object')

In [410]:
animal = pd.DataFrame()

In [469]:
Animals['Lat_Lon'] = "NA"

In [467]:
animal.head()

,SPART_ID,Scientific_Name,Common_Name,Threatened_Status,Class,Family,Description,Habitat,Feeding,Threats,Regions,Image,Profile,Lat_Lon
0,36,Balaenoptera musculus,Blue Whale,Endangered,Mammalia,Balaenopteridae,Both the Antarctic blue whale and pygmy blue w...,Blue whale habitat is variable between the two...,"In Antarctic waters, the blue whale feeds on k...",Several anthropogenic threats have been identi...,NT; CI; QLD; NSW; TAS; VIC; WA; ACI; CSI; SA; ...,Google Images http://www.yoto98.noaa.gov/books...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((153.22 -68.34999999999997, 153..."
1,40,Eubalaena australis,Southern Right Whale,Endangered,Mammalia,Balaenidae,The southern right whale is a large baleen wha...,Feeding HabitatThe feeding habitat of the sout...,Knowledge of southern right whale prey items h...,The Conservation Management Plan for the Sout...,QLD; NSW; TAS; VIC; WA; CSI; SA; HMI; AAT; JBT...,Google Images http://www.abc.net.au/storm/expo...,http://www.environment.gov.au/cgi-bin/sprat/pu...,MULTIPOLYGON (((159.0400000000001 -50.99999999...
2,77,Pseudomys shortridgei,"Heath Mouse, Dayang, Heath Rat",Endangered,Mammalia,Muridae,[],,,,VIC; WA; SA,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,MULTIPOLYGON (((141.1599999990001 -37.38999999...
3,88,Pseudomys fumeus,"Smoky Mouse, Konoom",Endangered,Mammalia,Muridae,The Konoom is an Australian native rodent with...,The Konoom occurs in a wide variety of habitat...,The Konoom has a seasonal diet that is based o...,Predation by introduced speciesThe relatively ...,ACT; NSW; VIC; SA,Google Images http://www.npws.nsw.gov.au/wildl...,http://www.environment.gov.au/cgi-bin/sprat/pu...,"MULTIPOLYGON (((143.58 -38.83999999999997, 143..."
4,98,Pseudomys oralis,"Hastings River Mouse, Koontoo",Endangered,Mammalia,Muridae,The Hastings River Mouse is a brownish-grey ro...,The main factors determining the species' pres...,The diet of the Hastings River Mouse has been ...,,QLD; NSW,Google Images http://www.npws.nsw.gov.au/wildl...,http://www.environment.gov.au/cgi-bin/sprat/pu...,MULTIPOLYGON (((151.3000000000001 -32.12999999...


In [451]:
for i in range(len(Animals)):
    st = Animals['Regions'][i]
    Animals['Regions'][i] = pd.eval(st)

C:\Users\V.PAVAN\Anaconda3\envs\rstudio\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [454]:
for i in range(len(Animals)):
    st = Animals['Regions'][i]
    Animals['Regions'][i] = '; '.join(map(str, st))

C:\Users\V.PAVAN\Anaconda3\envs\rstudio\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [465]:
Animals.merge(animal, how='right', indicator=True)

,SPART_ID,Scientific_Name,Common_Name,Threatened_Status,Class,Family,Description,Habitat,Feeding,Threats,Regions,Image,Profile,_merge
0,36,Balaenoptera musculus,Blue Whale,Endangered,Mammalia,Balaenopteridae,Both the Antarctic blue whale and pygmy blue w...,Blue whale habitat is variable between the two...,"In Antarctic waters, the blue whale feeds on k...",Several anthropogenic threats have been identi...,NT; CI; QLD; NSW; TAS; VIC; WA; ACI; CSI; SA; ...,Google Images http://www.yoto98.noaa.gov/books...,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
1,40,Eubalaena australis,Southern Right Whale,Endangered,Mammalia,Balaenidae,The southern right whale is a large baleen wha...,Feeding HabitatThe feeding habitat of the sout...,Knowledge of southern right whale prey items h...,The Conservation Management Plan for the Sout...,QLD; NSW; TAS; VIC; WA; CSI; SA; HMI; AAT; JBT...,Google Images http://www.abc.net.au/storm/expo...,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
2,77,Pseudomys shortridgei,"Heath Mouse, Dayang, Heath Rat",Endangered,Mammalia,Muridae,[],,,,VIC; WA; SA,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
3,88,Pseudomys fumeus,"Smoky Mouse, Konoom",Endangered,Mammalia,Muridae,The Konoom is an Australian native rodent with...,The Konoom occurs in a wide variety of habitat...,The Konoom has a seasonal diet that is based o...,Predation by introduced speciesThe relatively ...,ACT; NSW; VIC; SA,Google Images http://www.npws.nsw.gov.au/wildl...,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
4,98,Pseudomys oralis,"Hastings River Mouse, Koontoo",Endangered,Mammalia,Muridae,The Hastings River Mouse is a brownish-grey ro...,The main factors determining the species' pres...,The diet of the Hastings River Mouse has been ...,,QLD; NSW,Google Images http://www.npws.nsw.gov.au/wildl...,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,1047,Procellaria cinerea,Grey Petrel,NaN,Aves,Procellariidae,[],,,,TAS; CMA,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
546,78900,Tursiops aduncus (Arafura/Timor Sea populations),Spotted Bottlenose Dolphin (Arafura/Timor Sea ...,NaN,Mammalia,Delphinidae,[],,,,NT; WA; ACI; CMA,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
547,83000,Thalasseus bergii,Crested Tern,NaN,Aves,Laridae,[],,,,NaN,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only
548,80610,Cecropis daurica,Red-rumped Swallow,NaN,Aves,Hirundinidae,[],,,,NaN,Google Images,http://www.environment.gov.au/cgi-bin/sprat/pu...,right_only


In [461]:
s1 = pd.merge(Animals, animal, how='inner', on=['Scientific_Name'])

In [462]:
s1.columns

Index(['SPART_ID_x', 'Scientific_Name', 'Common_Name_x', 'Threatened_Status_x',
       'Class_x', 'Family_x', 'Description_x', 'Habitat_x', 'Feeding_x',
       'Threats_x', 'Regions_x', 'Image_x', 'Profile_x', 'SPART_ID_y',
       'Common_Name_y', 'Threatened_Status_y', 'Class_y', 'Family_y',
       'Description_y', 'Habitat_y', 'Feeding_y', 'Threats_y', 'Regions_y',
       'Image_y', 'Profile_y'],
      dtype='object')

In [259]:
dummy2 = pd.read_csv("ACT_animals.csv")

In [473]:
Animals.to_csv("ENDANGERED_AUSTRALIAN_ANIMALS.csv")

In [472]:
animal.to_csv("AUSTRALIA_ANIMALS.csv")

In [9]:
animals = pd.read_csv("AUSTRALIA_ANIMALS.csv")

In [112]:
maps = animals[['SPART_ID','Scientific_Name','geometry']]

In [10]:
animals = animals[['SPART_ID','Lat_Lon']]

In [137]:
animals.to_csv("final_maps_animals.csv",index=False)

In [132]:
animals = animals[['SPART_ID','Lat_Lon']]

In [92]:
import numpy as np
#animals = animals.replace(r'\[\]', "NA", regex=True)
animals['Threats'] = animals['Threats'].fillna('NA') 

In [134]:
animals.to_csv("final_maps_animals.csv")

In [81]:
animals.columns

Index(['SPART_ID', 'Scientific_Name', 'Threatened_Status', 'Class', 'Family',
       'Description', 'Habitat', 'Feeding', 'Threats', 'Profile'],
      dtype='object')

In [93]:
animals.head()

,SPART_ID,Scientific_Name,Threatened_Status,Class,Family,Description,Habitat,Feeding,Threats,Profile
0,36,Balaenoptera musculus,Endangered,Mammalia,Balaenopteridae,Both the Antarctic blue whale and pygmy blue w...,Blue whale habitat is variable between the two...,"In Antarctic waters, the blue whale feeds on k...",Several anthropogenic threats have been identi...,http://www.environment.gov.au/cgi-bin/sprat/pu...
1,40,Eubalaena australis,Endangered,Mammalia,Balaenidae,The southern right whale is a large baleen wha...,Feeding HabitatThe feeding habitat of the sout...,Knowledge of southern right whale prey items h...,The Conservation Management Plan for the Sout...,http://www.environment.gov.au/cgi-bin/sprat/pu...
2,77,Pseudomys shortridgei,Endangered,Mammalia,Muridae,NA,NA,NA,NA,http://www.environment.gov.au/cgi-bin/sprat/pu...
3,88,Pseudomys fumeus,Endangered,Mammalia,Muridae,The Konoom is an Australian native rodent with...,The Konoom occurs in a wide variety of habitat...,The Konoom has a seasonal diet that is based o...,Predation by introduced speciesThe relatively ...,http://www.environment.gov.au/cgi-bin/sprat/pu...
4,98,Pseudomys oralis,Endangered,Mammalia,Muridae,The Hastings River Mouse is a brownish-grey ro...,The main factors determining the species' pres...,The diet of the Hastings River Mouse has been ...,NA,http://www.environment.gov.au/cgi-bin/sprat/pu...


### Images Data

In [15]:
aus_animals = pd.read_csv("AUSTRALIA_ANIMALS.csv")

Index(['Unnamed: 0', 'SPART_ID', 'Scientific_Name', 'Common_Name',
       'Threatened_Status', 'Class', 'Family', 'Description', 'Habitat',
       'Feeding', 'Threats', 'Regions', 'Image', 'Profile', 'Lat_Lon'],
      dtype='object')

In [10]:
images = pd.read_csv("images_split_edit_animals.csv")

In [8]:
images["SPART_ID"] = aus_animals["SPART_ID"]

In [14]:
images.head()

,SPART_ID,Scientific_Name,Common_Name,images
0,36,Balaenoptera musculus,Blue Whale,['https://upload.wikimedia.org/wikipedia/commo...
1,40,Eubalaena australis,Southern Right Whale,['https://upload.wikimedia.org/wikipedia/commo...
2,77,Pseudomys shortridgei,"Heath Mouse, Dayang, Heath Rat",['https://upload.wikimedia.org/wikipedia/commo...
3,88,Pseudomys fumeus,"Smoky Mouse, Konoom",[]
4,98,Pseudomys oralis,"Hastings River Mouse, Koontoo",['https://upload.wikimedia.org/wikipedia/commo...


In [13]:
idx = 0
new_col = aus_animals["SPART_ID"]  # can be a list, a Series, an array or a scalar   
images.insert(loc=idx, column='SPART_ID', value=new_col)

In [17]:
images.to_csv("images_split_edit_animals.csv")

In [ ]:
import pandas as pd
import string

data = pd.read_csv('C:/Users/V.PAVAN/Desktop/FIT120 IE/project/work/images_split_edit_animals.csv')
data = data[data['images'] == "[]"] 

for index,row in data.iterrows():
    print('Sci_name: '+data['Scientific_Name'][index]+'\n')
    print('comm_name: '+data['Common_Name'][index]+'\n')
    word = 'enter'
    images_url = []
    while word != 'exit':
        word = input('Enter the url:')
        images_url.append(word)
    data['images'][index] = images_url
data.to_csv('hand_images.csv',index=False)
print('Done with the images:=)')

Sci_name: Balaenoptera musculus

comm_name: Blue Whale

Enter the url:https://en.wikipedia.org/wiki/Blue_whale#/media/File:Anim1754_-_Flickr_-_NOAA_Photo_Library.jpg
Enter the url:https://en.wikipedia.org/wiki/Blue_whale#/media/File:Blue_Whale_001_body_bw.jpg
Enter the url:https://en.wikipedia.org/wiki/Blue_whale#/media/File:Bluewhale2_noaa.jpg
Enter the url:https://en.wikipedia.org/wiki/Blue_whale#/media/File:Bluewhale1_noaa_crop.jpg
Enter the url:exit


C:\Users\V.PAVAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


Sci_name: Eubalaena australis

comm_name: Southern Right Whale

Enter the url:https://en.wikipedia.org/wiki/Southern_right_whale#/media/File:Southern_Right_Whale_(Eubalaena_australis)_(16358018502).jpg
Enter the url:https://en.wikipedia.org/wiki/Southern_right_whale#/media/File:Ballenafranca+alvina.jpg
Enter the url:exit
Sci_name: Pseudomys shortridgei

comm_name: Heath Mouse, Dayang, Heath Rat

Enter the url:https://live.staticflickr.com/812/40242103384_b1cb2cd6f5.jpg
Enter the url:exit
Sci_name: Pseudomys fumeus

comm_name: Smoky Mouse, Konoom

Enter the url:https://collections.museumvictoria.com.au/content/media/11/671911-small.jpg
Enter the url:exit
Sci_name: Pseudomys oralis

comm_name: Hastings River Mouse, Koontoo

Enter the url:https://en.wikipedia.org/wiki/Hastings_River_mouse#/media/File:Hastings_River_Mouse.jpg
Enter the url:exit
Sci_name: Pteropus conspicillatus

comm_name: Spectacled Flying-fox

Enter the url:https://en.wikipedia.org/wiki/Spectacled_flying_fox#/media/File:

Enter the url:https://en.wikipedia.org/wiki/Tristan_albatross#/media/File:Tristan_Albatross_(1).jpg
Enter the url:exit
Sci_name: Thalassarche chrysostoma

comm_name: Grey-headed Albatross

Enter the url:https://en.wikipedia.org/wiki/Grey-headed_albatross#/media/File:Thalassarche_chrysostoma_-_SE_Tasmania.jpg
Enter the url:https://en.wikipedia.org/wiki/Grey-headed_albatross#/media/File:Grey-headed_albatross_chick.jpg
Enter the url:exit
Sci_name: Botaurus poiciloptilus

comm_name: Australasian Bittern

Enter the url:https://en.wikipedia.org/wiki/Australasian_bittern#/media/File:Australasian_Bittern_(cropped).jpg
Enter the url:https://en.wikipedia.org/wiki/Australasian_bittern#/media/File:Australasian_Bittern_(Botaurus_poiciloptilus)_in_the_grass.jpg
Enter the url:exit
Sci_name: Fregata andrewsi

comm_name: Christmas Island Frigatebird, Andrew's Frigatebird

Enter the url:https://en.wikipedia.org/wiki/Christmas_frigatebird#/media/File:Christmas_Island_Frigatebird_male_-_Jakarta_Bay,_Indon

Enter the url:https://en.wikipedia.org/wiki/Buff-banded_rail#/media/File:Buff-banded_Rail_Fafa_Island.jpg
Enter the url:exit
Sci_name: Chalcophaps indica natalis

comm_name: Christmas Island Emerald Dove, Emerald Dove (Christmas Island)

Enter the url:https://en.wikipedia.org/wiki/Common_emerald_dove#/media/File:Chalcophaps_indica_-a_pair_in_captivity-8a.jpg
Enter the url:https://en.wikipedia.org/wiki/Common_emerald_dove#/media/File:Common_Emerald_Dove.jpg
Enter the url:exit
Sci_name: Cyanoramphus cookii

comm_name: Norfolk Island Green Parrot, Tasman Parakeet, Norfolk Island Parakeet



In [3]:
full_data = pd.read_csv('images_split_edit_animals.csv')


full_data = full_data[full_data['images']!='[]'].reset_index().drop(['index'],axis=1)

In [5]:
hand_images = pd.read_csv('hand_images.csv')

def correct_images(x):
    images_list = x.split(',')
    return_images = []
    for value in images_list:
        match = re.search("(?P<url>https?://[^\s]+)", value)
        if match:
            return_images.append(match.group("url"))
        else:
            continue
    return return_images

In [8]:
hand_images['images'] = hand_images.apply(lambda x: correct_images(x['images']),axis=1)


pd.concat([full_data,hand_images]).reset_index().drop(['index'],axis=1).to_csv('images_final.csv',index=False)


In [8]:
import pandas as pd

In [9]:
final_images = pd.read_csv('images_final.csv')

In [13]:
final_images.columns

Index(['SPART_ID', 'Scientific_Name', 'Common_Name', 'images'], dtype='object')

In [15]:
type(final_images['SPART_ID'][0])

numpy.int64

### Map Data 

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, mapping, Polygon, MultiPolygon, Point

In [3]:
shape_file = gpd.read_file("snes_public_grids_08Aug2019_shapefile/snes_species_combined.shp")
image_file = pd.read_csv("images_final.csv")

In [4]:
shape_file.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'listed_id', 'map_id',
       'sci_name', 'comm_name', 'threatened', 'migratory', 'pres_rank',
       'category', 'extracted', 'tax_group', 'tax_family', 'tax_order',
       'tax_class', 'tax_phylum', 'tax_king', 'other_ids', 'cell_size',
       'regions', 'attributio', 'sprat_prof', 'geometry'],
      dtype='object')

In [5]:
image_file['lower_name'] = image_file['Scientific_Name'].str.lower()
shape_file['lower_name'] = shape_file['sci_name'].str.lower()
#Performing the inner join
locations = pd.merge(shape_file[['lower_name','geometry']],image_file[['lower_name','SPART_ID']],on='lower_name',how='inner')

In [6]:
new_df = gpd.GeoDataFrame(locations,geometry = "geometry")
new_df.crs = {'init':'epsg:4326'}

In [7]:
outdf = gpd.GeoDataFrame(columns=new_df.columns)
for idx,row in new_df.iterrows():
    #print(row)
    if type(row.geometry) == MultiPolygon:
        multdf = gpd.GeoDataFrame(columns = new_df.columns)
        recs = len(row.geometry)
        multdf = multdf.append([row]*recs,ignore_index=True)
        for geom in range(recs):
            multdf.loc[geom,'geometry'] = row.geometry[geom]
        outdf = outdf.append(multdf,ignore_index=True)
    if type(row.geometry) == Polygon:
        outdf = outdf.append(row,ignore_index=True)

In [8]:
new_df=gpd.GeoDataFrame(outdf,geometry='geometry')#adding the point objects as a column to the dataframe.
new_df.crs={'init' :'epsg:4326'}


new_df = new_df.drop(['lower_name'], axis = 1)

In [ ]:
#new_df.to_csv("maps_edit_animals.csv")

## Inserting into Database:

In [18]:
states = aus_animals[['SPART_ID','Regions']]

In [7]:
import pymysql
mydb = pymysql.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="services"
)

In [30]:
cursor.close
mydb.close

<bound method Connection.close of <pymysql.connections.Connection object at 0x00000184CA4E57B8>>

In [ ]:
cursor = mydb.cursor()
sql = "INSERT INTO animals_states(SPART_ID, States) VALUES (%s, %s)"
cursor.executemany(sql, tuples)
mydb.commit()

In [9]:
animals = new_df[['SPART_ID', 'geometry']]

In [10]:
animals['geometry'] = animals['geometry'].astype(str)

In [5]:
data = services
subset = data[['SPART_ID', 'geometry']]
tuples = [tuple(x) for x in subset.to_numpy()]

In [29]:
states.to_csv("animal_states.csv")